# Exercise 1: Reddit API Data Collection

Librerías

In [20]:
import praw
import pandas as pd
import os
from dotenv import load_dotenv

#### Part 1: Reddit API Setup

In [21]:
reddit = praw.Reddit(
    client_id="Zq-6-kuY12svcQqLLtyAJg",
    client_secret="ezV1Ym44_cTogMC25nxUvAJuuRRbqw",
    user_agent=f"Python:webscrapingProject:v1.0 (by /u/QLAB)",
)

#### Part 2: Collect Data and Storage

**Parametros**  
Definición de subreddits número de posts

In [22]:
# subreddits
subReddits = ["politics", "PoliticalDiscussion", "worldnews"]

# cantidad de posts
nPost = 20

**Obtención de Datos**

In [24]:
DATA = [] # list de resultados

for sub in subReddits:  # por cada subreddit
    subreddit = reddit.subreddit(sub)

    # Top posts
    for post in subreddit.top(limit=nPost):
        DATA.append({
        "subreddit": sub,
        "category": "top",
        "title": post.title,
        "score": post.score,
        "num_comments": post.num_comments,
        "id": post.id,
        "url": post.url
    })
dfPost = pd.DataFrame(DATA) 
print(dfPost.head())

# Exportar
dfPost.to_csv("./out/reddit_posts.csv", index=False, encoding="utf-8")

  subreddit category                                              title  \
0  politics      top  Megathread: Joe Biden Projected to Defeat Pres...   
1  politics      top  Mitch McConnell Will Lose Control Of The Senat...   
2  politics      top  Megathread: House Votes to Impeach President D...   
3  politics      top  Trump Threatens to ‘Leave the Country’ if He L...   
4  politics      top  Demands for Kushner to Resign Over 'Staggering...   

    score  num_comments      id  \
0  214313         80928  jptq5n   
1  156756         10099  krntg6   
2  147733         50628  ecm1zg   
3  135302         16093  jcm5dz   
4  129739          6746  i19sjg   

                                                 url  
0  https://www.reddit.com/r/politics/comments/jpt...  
1  https://www.buzzfeednews.com/article/paulmcleo...  
2  https://www.reddit.com/r/politics/comments/ecm...  
3  https://www.thedailybeast.com/trump-threatens-...  
4  https://www.commondreams.org/news/2020/07/31/d...  


**Obtención de Comentarios**

2. El primer for recorre los posts que se tienen en post_data. Luego se crea (y obtiene desde la API) un objeto Submission de PRAW para cada postc usando su id; este objeto tiene atributos y métodos que describen y permiten interactuar con el post.

Utilizamos el atributo comments del objeto post para acceder a los comentarios del post. Especificamos el método replace_more() para que los placeholders se reemplacen por comentarios reales. Indicamos limit=0 para no solicitar que se descarguen más comentarios anidados, es decir, cargamos solo los comentarios principales

El segundo for itera para cada comentario de los 5 primeros comentarios
- [:5] toma los primeros 5 comentarios de primer nivel según el orden actual de comentarios (“best” es el orden predeterminado, mezcla de relevancia + votos)

In [ ]:
commentDATA = [] # lista para resultados

for post_info in DATA:    
    post = reddit.submission(id=post_info["id"])    
    post.comments.replace_more(limit=0) # 0: solo comentario principal
    
    for comm in post.comments[:5]:
        commentDATA.append({
            "post_id": post.id,
            "body": comm.body,
            "score": comm.score
        })

dfComments = pd.DataFrame(commentDATA)

# Exportar
dfComments.to_csv("./out/reddit_comments.csv", index=False, encoding="utf-8")

print(dfComments.head())

  post_id                                               body  score
0  jptq5n  EVERYONE WHO TURNED OUT IN 2020 NEEDS TO COME ...   2910
1  jptq5n  Never won the popular vote, got impeached, los...   2682
2  jptq5n  As German I understand how a country can be sc...   2446
3  jptq5n  Let us not soon forget that this victory was n...  10374
4  jptq5n  Trump is living the 2020 experience - got Covi...   6071


**Merge**  
Linkear cada comentario su parent post.

In [ ]:
# Merge postsDF-commentDF
dfMerged = dfComments.merge(dfPost, left_on="post_id", right_on="id", how="left")

# Exportar
dfMerged.to_csv("./out/comments_linked_posts.csv", index=False, encoding="utf-8")
dfMerged.head()

,post_id,body,score_x,subreddit,category,title,score_y,num_comments,id,url
0,jptq5n,EVERYONE WHO TURNED OUT IN 2020 NEEDS TO COME ...,2910,politics,top,Megathread: Joe Biden Projected to Defeat Pres...,214313,80928,jptq5n,https://www.reddit.com/r/politics/comments/jpt...
1,jptq5n,"Never won the popular vote, got impeached, los...",2682,politics,top,Megathread: Joe Biden Projected to Defeat Pres...,214313,80928,jptq5n,https://www.reddit.com/r/politics/comments/jpt...
2,jptq5n,As German I understand how a country can be sc...,2446,politics,top,Megathread: Joe Biden Projected to Defeat Pres...,214313,80928,jptq5n,https://www.reddit.com/r/politics/comments/jpt...
3,jptq5n,Let us not soon forget that this victory was n...,10374,politics,top,Megathread: Joe Biden Projected to Defeat Pres...,214313,80928,jptq5n,https://www.reddit.com/r/politics/comments/jpt...
4,jptq5n,Trump is living the 2020 experience - got Covi...,6071,politics,top,Megathread: Joe Biden Projected to Defeat Pres...,214313,80928,jptq5n,https://www.reddit.com/r/politics/comments/jpt...
